In [8]:
# Gautam Jain, Jannis Horn 

%matplotlib notebook
import time
import os
from typing import Tuple
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
from mpl_toolkits.axes_grid1 import make_axes_locatable
import torch  
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as topt
import wandb
import cv2
import os
from torchvision.transforms import ToTensor
import torchvision.models as models
from torchviz import make_dot
from array2gif import write_gif

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

torch.manual_seed( 666 )
print(device)

cuda


In [9]:
class Logger:
    
    def __init__( self, prefix ):
        self.cur_ep = 0
        self.loss = []
        self.losses_pim = [[],[],[]]
        self.time = []
        self.prefix = prefix
        
    def plot( self, loss, loss_pim, time ):
        self.cur_ep += 1
        self.loss.append( loss )
        self.losses_pim[0].append( loss_pim[0] )
        self.losses_pim[1].append( loss_pim[1] )
        self.losses_pim[2].append( loss_pim[2] )
        self.time.append( time )
        self._log( self.loss, "{}_Loss".format( self.prefix ), ["epoch", "Loss"] )
        self._log( self.time, "{}_Time".format( self.prefix ), ["epoch", "Time"] )
        self._log( self.losses_pim[0], "{}_Loss Im1".format( self.prefix ), ["epoch", "Loss"] )
        self._log( self.losses_pim[1], "{}_Loss Im2".format( self.prefix ), ["epoch", "Loss"] )
        self._log( self.losses_pim[2], "{}_Loss Im3".format( self.prefix ), ["epoch", "Loss"] )
        
    def _log( self, dt, fig_name, col_names ):
        data = zip( range( self.cur_ep ), dt )
        table = wandb.Table( data=data, columns = col_names )
        wandb.log( {fig_name: wandb.plot.line( table, col_names[0], col_names[1], title ) } )
        
    def plotImages( self, teacher_ims, out_ims, fps=2 ):
        t_gif = np.stack( [self._drawOutline(im) for im in teacher_ims ], axis=0 )
        o_gif = np.stack( [self._drawOutline(im) for im in out_ims ], axis=0 )
        self._logGif( "{}_real".format( self.prefix ), t_gif, fps )
        self._logGif( "{}_out".format( self.prefix ), o_gif, fps )
        
    def _drawOutline( self, im, clr, outline_size ):
        pad_im = np.empty( [im.shape[0],
                            im.shape[1]+2*outline_size,
                            im.shape[2]+2*outline_size] )
        for ch in range(3):
            pad_im[ch,:,:] = np.pad( im[ch,:,:], pad_width=outline_size, mode='constant', constant_value=clr[ch] )
        return pad_im
    
    def _logGif( self, key, ims, fps ):
        wandb.log( {key: wandb.Video( ims, fps=fps )}, 
                   step=epoch )
        
        
        

def plot( epoch, loss, loss_pim, tr_time, ims ):
    wandb.log( {"loss": dst_loss,
                "gen_loss": gen_loss,
                "sample": [wandb.Image(ims, caption="Sample Image")],
                "tr_time": tr_time},
                step=epoch )

In [4]:
#convGRU code taken from: https://github.com/SreenivasVRao/ConvGRU-ConvLSTM-PyTorch/blob/master/convgru.py
from convgru import ConvGRU

#DSSIM Loss taken from: https://github.com/Po-Hsun-Su/pytorch-ssim/blob/master/pytorch_ssim/__init__.py
from dssim import SSIM

In [5]:
class LadderLayer( nn.Module ):
    def __init__( self, w, h, in_channels, out_channels, use_loc_dep ):
        super( LadderLayer, self ).__init__()
        if not use_loc_dep: self.conv = nn.Conv2d( in_channels, out_channels, (1,1) )
        else: self.conv = LocationAwareConv2d( True, False, w, h, in_channels, out_channels, (1,1) )
        self.conv_gru3 = ConvGRU( in_channels, out_channels, (3,3), 1, batch_first=True )
        self.conv_gru5 = ConvGRU( in_channels, out_channels, (5,5), 1, batch_first=True )
        self.conv_gru7 = ConvGRU( in_channels, out_channels, (7,7), 1, batch_first=True )
        
    def forward( self, x ):
        out_l = []
        for it in range( x.shape[1] ):
            out_l.append( self.conv( x[:,it,:,:,:] ) )
        out = torch.stack( out_l, dim=1 )
        out3,_ = self.conv_gru3( x )
        out5,_ = self.conv_gru5( x )
        out7,_ = self.conv_gru7( x )
        return torch.cat( [out, out3[0], out5[0], out7[0]], dim=2 )
        
        
class ReconstructionLayer( nn.Module ):
    def __init__( self, in_channels, use_btnorm ):
        super( ReconstructionLayer, self ).__init__()
        if not use_btnorm: self.relu = nn.ReLU()
        else: self.relu = nn.Sequential( nn.ReLU(), nn.BatchNorm2d( in_channels ) )
        self.conv1 = nn.Conv2d( in_channels, 1024, (3,3), padding=1 )
        self.shuffle = nn.PixelShuffle( 2 )
        self.conv2 = nn.Conv2d( 256, 64, (3,3), padding=1 )
        
    def forward( self, x ):
        out = self.relu( x )
        out = self.conv1( out )
        out = self.shuffle( out )
        out = self.conv2( out )
        return out
        
        
class LocationAwareConv2d(torch.nn.Conv2d):
    def __init__(self,locationAware,gradient,w,h,in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True):
        super().__init__(in_channels, out_channels, kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        if locationAware:
            self.locationBias=torch.nn.Parameter(torch.zeros(w,h,3))
            self.locationEncode=torch.autograd.Variable(torch.ones(w,h,3))
            if gradient:
                for i in range(w):
                    self.locationEncode[i,:,1]=self.locationEncode[:,i,0]=i/float(w-1)
        
        self.up=torch.nn.Upsample(size=(w,h), mode='bilinear', align_corners=False)
        self.w=w
        self.h=h
        self.locationAware=locationAware
        
    def forward(self,inputs):
        if self.locationAware:
            if self.locationBias.device != inputs.device:
                self.locationBias=self.locationBias.to(inputs.get_device())
            if self.locationEncode.device != inputs.device:
                self.locationEncode=self.locationEncode.to(inputs.get_device())
            b=self.locationBias*self.locationEncode
        convRes=super().forward(inputs)
        if convRes.shape[2]!=self.w and convRes.shape[3]!=self.h:
            convRes=self.up(convRes)
        if self.locationAware:
            return convRes+b[:,:,0]+b[:,:,1]+b[:,:,2]
        else:
            return convRes
        
    def  __str__( self ):
        return( "LocationAware{}, LocAware={}, gradient={}".format( super().__str__(), self.locationAware, self.Gradient ) )

In [7]:
class PredictionModel( nn.Module ):
    
    def __init__( self, w, h ):
        super( PredictionModel, self ).__init__()
        self.resnet18 = models.resnet18( pretrained=True )
        for param in self.resnet18.parameters():
            param.requires_grad = False
        self.ladders = nn.ModuleDict( {
            "ladder1": LadderLayer( int(w/2), int(h/2), 64, 64, False ),
            "ladder2": LadderLayer( int(w/4), int(h/4), 64, 64, False ),
            "ladder3": LadderLayer( int(w/8), int(h/8), 128, 64, True ),
            "ladder4": LadderLayer( int(w/16), int(h/16), 256, 64, True ) 
        } )
        self.recons = nn.ModuleDict( {
            "recon1": ReconstructionLayer( 64*5, True ),
            "recon2": ReconstructionLayer( 64*5, True ),
            "recon3": ReconstructionLayer( 64*5, True ),
            "recon4": ReconstructionLayer( 512, False )
        } )
        self.out_conv = nn.Conv2d( 320, 12, (1,1) )
        self.out_shuffle = nn.PixelShuffle( 2 )
        self.out_act = nn.Sigmoid()
        
        
    def getResnetOutputs( self, x ):
        inp_c = self.resnet18.conv1( x )
        inp = self.resnet18.bn1( inp_c )
        inp = self.resnet18.relu( inp )
        inp = self.resnet18.maxpool( inp )
        res_out1 = self.resnet18.layer1( inp )
        res_out2 = self.resnet18.layer2( res_out1 )
        res_out3 = self.resnet18.layer3( res_out2 )
        res_out4 = self.resnet18.layer4( res_out3 )
        return ( inp_c, res_out1, res_out2, res_out3, res_out4 )
    
    def getReconstructionOutput( self, res_x, rec_x, l_it ):
        l_out = self.ladders["ladder{}".format(l_it)]( res_x )
        r_out_l = []
        for it in range( rec_x.shape[1] ):
            r_out_l.append( self.recons["recon{}".format(l_it)]( rec_x[:,it,:,:,:] ) )
        r_out = torch.stack( r_out_l, dim=1 )
        out = torch.cat( [r_out, l_out], dim=2 )
        return out
    
    def getL2Norm( self ):
        out = torch.tensor(0.0).to( self.out_conv.weight.device )
        for param in self.ladders.parameters():
            out += torch.norm( param, 2 )
        for param in self.recons.parameters():
            out += torch.norm( param, 2 )
        out += torch.norm( self.out_conv.weight, 2 )
        return out
        
    def forward( self, x ):
        res_out = []
        for it in range( x.shape[1] ):
            res_out.append( self.getResnetOutputs( x[:,it,:,:,:] ) )
        res_0 = torch.stack( [r[0] for r in res_out], dim=1 )
        res_1 = torch.stack( [r[1] for r in res_out], dim=1 )
        res_2 = torch.stack( [r[2] for r in res_out], dim=1 )
        res_3 = torch.stack( [r[3] for r in res_out], dim=1 )
        res_4 = torch.stack( [r[4] for r in res_out], dim=1 )
        out = self.getReconstructionOutput( res_3, res_4, 4 )
        out = self.getReconstructionOutput( res_2, out, 3 )
        out = self.getReconstructionOutput( res_1, out, 2 )
        out = self.getReconstructionOutput( res_0, out, 1 )
        out_l = []
        for it in range( x.shape[1] ):
            out_f = self.out_conv(out[:,it,:,:,:])
            out_f = self.out_shuffle( out_f )
            out_f = self.out_act( out_f )
            out_l.append( out_f )
            
        return torch.stack( out_l, dim=1 )
        

model = PredictionModel(320,224)
#print(model.ladders["ladder1"], model.recons["recon1"])
inp = torch.rand( 2,6,3,320,224 ).to( device )
model = model.to( device )
out = model(inp)
print( out.shape )
print( model.getL2Norm() )
print(model)
make_dot( out, params=dict(model.named_parameters()) )

torch.Size([2, 6, 3, 320, 224])
tensor(339.0722, device='cuda:0', grad_fn=<AddBackward0>)
PredictionModel(
  (resnet18): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding

In [11]:
# Hyper params
name = "MNIST_Base"
lr = 1e-4
l2_lambda = 1e-4
num_epochs = 1000
ssim_win_size = 11
bt_size = 32

In [ ]:
# Dataset


In [ ]:
model = PredictionModel(dset.width,dset.height)
model = model.to( device )
optimizer = topt.Adam( model.parameters(), lr=lr )
loss_func = SSIM( window_size=ssim_win_size )
run = wandb.init( project="project_predictvideo", entity="cudavisionlab", name=name, reinit=True )
tr_logger = Logger( "train" )
ts_logger = Logger( "test" )

with run:
    wandb.config.lr = lr
    wandb.config.l2 = l2_lambda
    wandb.config.ssim_ws = ssim_win_size
    for epoch in range( num_epochs ):
        st_pt = time.time()
        bt_loss = np.array( 0 )
        losses = np.zeros( 3 )
        model.train()
        for x, teacher in train_loader:
            
            x, teacher = x.to( device ), teacher.to( device )
            optimizer.zero_grad()
            out = model( x )
            
            loss = torch.Tensor(0).to( device )
            for im_it in range( 3,6 ):
                cur_loss = loss_func( teacher[:,im_it,:,:,:], out[:,im_it,:,:,:] )
                loss += cur_loss
                losses[im_it] += cur_loss.detach().clone().cpu().item()
            loss += model.getL2Norm() *l2_lambda
            loss.backward()
            bt_loss += loss.detach().clone().cpu().item()
            optimizer.step()
            
        tr_logger.plot( bt_loss /len(train_loader), losses /len(train_loader), time.time() -st_pt )
        
        st_pt = time.time()
        bt_loss = np.array( 0 )
        losses = np.zeros( 3 )
        model.test()
        for x, teacher in test_loader:
            
            x, teacher = x.to( device ), teacher.to( device )
            out = model( x )
            
            loss = torch.Tensor(0).to( device )
            for im_it in range( 3,6 ):
                cur_loss = loss_func( teacher[:,im_it,:,:,:], out[:,im_it,:,:,:] )
                loss += cur_loss
                losses[im_it] += cur_loss.detach().clone().cpu().item()
            bt_loss += loss.detach().clone().cpu().item()
            
        ts_logger.plot( bt_loss /len(test_loader), losses /len(test_loader), time.time() -st_pt )
        ts_logger.plotImages( teacher[0,:,:,:,:].cpu().item(), out[0,:,:,:,:].cpu().item() )
        
    torch.save(model.state_dict(), "./project/nets/{}".format( name ) )